<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Assistants_API_overview_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assistants API Overview (Python SDK)

## API Chat Completions so với API Assistants

Các thành phần cơ bản của **API Chat Completions** là `Messages`, trên đó bạn thực hiện một `Completion` với một `Model` (`gpt-3.5-turbo`, `gpt-4`, v.v.). Nó nhẹ và mạnh mẽ, nhưng vốn dĩ không có trạng thái, có nghĩa là bạn phải quản lý trạng thái cuộc hội thoại, định nghĩa công cụ, tài liệu truy xuất và thực thi mã theo cách thủ công.

Các thành phần cơ bản của **API Assistants** là

- `Assistants`, bao gồm một mô hình cơ sở, các hướng dẫn, công cụ và tài liệu (ngữ cảnh),
- `Threads`, đại diện cho trạng thái của một cuộc hội thoại, và
- `Runs`, thực thi một `Assistant` trên một `Thread`, bao gồm các phản hồi dạng văn bản và sử dụng công cụ nhiều bước.

Chúng ta sẽ xem xét cách những thành phần này có thể được sử dụng để tạo ra các trải nghiệm mạnh mẽ, có trạng thái.


## Setup

### Python SDK

> **Note**
> We've updated our [Python SDK](https://github.com/openai/openai-python) to add support for the Assistants API, so you'll need to update it to the latest version (`1.2.3` at time of writing).


In [ ]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


And make sure it's up to date by running:


In [ ]:
!pip show openai | grep Version

Version: 1.2.3


### Pretty Printing Helper


In [ ]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Complete Example with Assistants API


### Assistants


You can also create Assistants directly through the Assistants API, like so:


In [ ]:
from openai import OpenAI
import os
from google.colab import userdata


client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))


assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo",
)
show_json(assistant)

{'id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'created_at': 1710933670,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

Regardless of whether you create your Assistant through the Dashboard or with the API, you'll want to keep track of the Assistant ID. This is how you'll refer to your Assistant throughout Threads and Runs.


Next, we'll create a new Thread and add a Message to it. This will hold the state of our conversation, so we don't have re-send the entire message history each time.


### Threads


Create a new thread:


In [ ]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo',
 'created_at': 1710933682,
 'metadata': {},
 'object': 'thread'}

Then add the Message to the thread:


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_FzkHxehXmub1ucR7mFSHJTNJ',
 'assistant_id': None,
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1710933689,
 'file_ids': [],
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo'}

> **Note**
> Even though you're no longer sending the entire history each time, you will still be charged for the tokens of the entire conversation history with each Run.


### Runs

Notice how the Thread we created is **not** associated with the Assistant we created earlier! Threads exist independently from Assistants, which may be different from what you'd expect if you've used ChatGPT (where a thread is tied to a model/GPT).

To get a completion from an Assistant for a given Thread, we must create a Run. Creating a Run will indicate to an Assistant it should look at the messages in the Thread and take action: either by adding a single response, or using tools.

> **Note**
> Runs are a key difference between the Assistants API and Chat Completions API. While in Chat Completions the model will only ever respond with a single message, in the Assistants API a Run may result in an Assistant using one or multiple tools, and potentially adding multiple messages to the Thread.

To get our Assistant to respond to the user, let's create the Run. As mentioned earlier, you must specify _both_ the Assistant and the Thread.


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_F2mOduFP47x8PyRVm9BT2VO3',
 'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710933731,
 'expires_at': 1710934331,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo',
 'tools': [],
 'usage': None}

Unlike creating a completion in the Chat Completions API, **creating a Run is an asynchronous operation**. It will return immediately with the Run's metadata, which includes a `status` that will initially be set to `queued`. The `status` will be updated as the Assistant performs operations (like using tools and adding messages).

To know when the Assistant has completed processing, we can poll the Run in a loop. (Support for streaming is coming soon!) While here we are only checking for a `queued` or `in_progress` status, in practice a Run may undergo a [variety of status changes](https://platform.openai.com/docs/api-reference/runs/object#runs/object-status) which you can choose to surface to the user. (These are called Steps, and will be covered later.)


In [ ]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_F2mOduFP47x8PyRVm9BT2VO3',
 'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'cancelled_at': None,
 'completed_at': 1710933732,
 'created_at': 1710933731,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1710933731,
 'status': 'completed',
 'thread_id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo',
 'tools': [],
 'usage': {'completion_tokens': 32, 'prompt_tokens': 48, 'total_tokens': 80}}

### Messages


Now that the Run has completed, we can list the Messages in the Thread to see what got added by the Assistant.


In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_1sFioGtpvSuSR6bMy78Bie3x',
   'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'Sure! Subtract 11 from both sides to get `3x = 3`, then divide by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1710933732,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_F2mOduFP47x8PyRVm9BT2VO3',
   'status': None,
   'thread_id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo'},
  {'id': 'msg_FzkHxehXmub1ucR7mFSHJTNJ',
   'assistant_id': None,
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1710933689,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'us

As you can see, Messages are ordered in reverse-chronological order – this was done so the most recent results are always on the first `page` (since results can be paginated). Do keep a look out for this, since this is the opposite order to messages in the Chat Completions API.


Let's ask our Assistant to explain the result a bit further!


In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_sWF1P2iCOlqgnJjK8EfPeImP',
   'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! To solve the equation `3x + 11 = 14`, you first isolate the term with the variable by subtracting 11 from both sides, then divide by the coefficient of x to find its value.'},
     'type': 'text'}],
   'created_at': 1710933830,
   'file_ids': [],
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_OJ2NNZlRxcz1OLDxnossxxVP',
   'status': None,
   'thread_id': 'thread_CvLO0Vou6c1yCHqOJXHqsBeo'}],
 'object': 'list',
 'first_id': 'msg_sWF1P2iCOlqgnJjK8EfPeImP',
 'last_id': 'msg_sWF1P2iCOlqgnJjK8EfPeImP',
 'has_more': False}

This may feel like a lot of steps to get a response back, especially for this simple example. However, you'll soon see how we can add very powerful functionality to our Assistant without changing much code at all!


### Example


Let's take a look at how we could potentially put all of this together. Below is all the code you need to use an Assistant you've created.

Since we've already created our Math Assistant, I've saved its ID in `MATH_ASSISTANT_ID`. I then defined two functions:

- `submit_message`: create a Message on a Thread, then start (and return) a new Run
- `get_response`: returns the list of Messages in a Thread


In [ ]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

I've also defined a `create_thread_and_run` function that I can re-use (which is actually almost identical to the [`client.beta.threads.create_and_run`](https://platform.openai.com/docs/api-reference/runs/createThreadAndRun) compound function in our API ;) ). Finally, we can submit our mock user requests each to a new Thread.

Notice how all of these API calls are asynchronous operations; this means we actually get async behavior in our code without the use of async libraries! (e.g. `asyncio`)


In [ ]:
id = MATH_ASSISTANT_ID

def create_thread_and_run(user_input, id):
    thread = client.beta.threads.create()
    run = submit_message(id, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?",id
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?",id)
thread3, run3 = create_thread_and_run("I don't like math. What can I do?",id)

# Now all Runs are executing...

Once all Runs are going, we can wait on each and get the responses.


In [ ]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: Sure! Subtract 11 from both sides to get `3x = 3`, then divide by 3 to find `x = 1`.

# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is a branch of mathematics that focuses on the study of vectors, vector spaces, and linear transformations.

# Messages
user: I don't like math. What can I do?
assistant: Try to find the fun in math by solving real-life problems or playing math-related games.

# Messages
user: I don't like math. What can I do?
assistant: Try to find the fun in math by solving real-life problems or playing math-related games.
user: Thank you!
assistant: You're welcome! If you have any more questions, feel free to ask.



Et voilà!

You may have noticed that this code is not actually specific to our math Assistant at all... this code will work for any new Assistant you create simply by changing the Assistant ID! That is the power of the Assistants API.


## Tools

A key feature of the Assistants API is the ability to equip our Assistants with Tools, like Code Interpreter, Retrieval, and custom Functions. Let's take a look at each.

### Code Interpreter

Let's equip our Math Tutor with the [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) tool, which we can do from the Dashboard...


...or the API, using the Assistant ID.


In [ ]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'created_at': 1710933670,
 'description': None,
 'file_ids': ['file-nfw91nJE2kbVTVHJCKBxBmqx'],
 'instructions': 'You are a math tutor.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

Now, let's ask the Assistant to use its new tool.


In [ ]:
thread, run = create_thread_and_run(
    "Generate the first 20 fibbonaci numbers with code.", id = MATH_ASSISTANT_ID
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Generate the first 20 fibbonaci numbers with code.
assistant: I have generated the first 20 Fibonacci numbers. Here they are:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]



And that's it! The Assistant used Code Interpreter in the background, and gave us a final response.

For some use cases this may be enough – however, if we want more details on what precisely an Assistant is doing we can take a look at a Run's Steps.

### Steps


A Run is composed of one or more Steps. Like a Run, each Step has a `status` that you can query. This is useful for surfacing the progress of a Step to a user (e.g. a spinner while the Assistant is writing code or performing retrieval).


In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [ ]:
run_steps.data

[RunStep(id='step_GCXlwEfHT2s0yksZfswhBmk9', assistant_id='asst_wIiQdreLlLNzoud1MxL2iheM', cancelled_at=None, completed_at=1710936027, created_at=1710936024, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_WSG2j8XD37GlXTNaW5Pq9Met', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_oHTHdOm1w7tT8ZYg1k9zwhs0', code_interpreter=CodeInterpreter(input='# Function to generate the first n Fibonacci numbers\ndef generate_fibonacci(n):\n    fibonacci_numbers = [0, 1]\n    for i in range(2, n):\n        next_number = fibonacci_numbers[-1] + fibonacci_numbers[-2]\n        fibonacci_numbers.append(next_number)\n    return fibonacci_numbers\n\n# Generate the first 20 Fibonacci numbers\nfirst_20_fibonacci = generate_fibonacci(20)\nfirst_20_fibonacci', outputs=[CodeInterpreterOutputLogs(logs='[0,\n 1,\n 1,\n 2,\n 3,\n 5,\n 8,\n 13,\n 21,\n 34,\n 55,\n 89,\n 144,\n 233,\n 377,\n 610,\n 987,\n 1597,\

Let's take a look at each Step's `step_details`.


In [ ]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_WQNwrTIpqNApEfAbhre6tIVD',
   'code_interpreter': {'input': 'def fibonacci(n):\n    fib_seq = [0, 1]\n    for i in range(2, n):\n        fib_seq.append(fib_seq[i-1] + fib_seq[i-2])\n    return fib_seq\n\n# Generate the first 20 Fibonacci numbers\nfibonacci(20)',
    'outputs': [{'logs': '[0,\n 1,\n 1,\n 2,\n 3,\n 5,\n 8,\n 13,\n 21,\n 34,\n 55,\n 89,\n 144,\n 233,\n 377,\n 610,\n 987,\n 1597,\n 2584,\n 4181]',
      'type': 'logs'}]},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_g5buy2IXPziCsoH0Y3cG38IL'},
 'type': 'message_creation'}

null


We can see the `step_details` for two Steps:

1. `tool_calls` (plural, since it could be more than one in a single Step)
2. `message_creation`

The first Step is a `tool_calls`, specifically using the `code_interpreter` which contains:

- `input`, which was the Python code generated before the tool was called, and
- `output`, which was the result of running the Code Interpreter.

The second Step is a `message_creation`, which contains the `message` that was added to the Thread to communicate the results to the user.


### Retrieval

Another powerful tool in the Assistants API is [Retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval): the ability to upload files that the Assistant will use as a knowledge base when answering questions. This can also be enabled from the Dashboard or the API, where we can upload files we want to be used.


![Enabling retrieval](../images/assistants_overview_enable_retrieval.png)


In [ ]:
# Upload the file
file = client.files.create(
    file=open(
        "/content/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'created_at': 1710933670,
 'description': None,
 'file_ids': ['file-u5H0QWd95vzYHIEmJfR1Vn3E'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [ ]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences.", id = MATH_ASSISTANT_ID
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What are some cool math concepts behind this ML paper pdf? Explain in two sentences.
assistant: This paper explores the concept of language models being trained on large amounts of text data to perform various natural language processing tasks without explicit supervision, showcasing the potential of zero-shot task transfer by conditioning on both input and task. The approach leverages the capacity of language models to improve performance across tasks in a log-linear fashion, demonstrating the importance of multitask learning and generalization in machine learning systems.



In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_4eoK0vY3G3YNMih7MWg9Zpp1',
   'retrieval': {},
   'type': 'retrieval'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_QNaNvRvLWiJsHAoedRlRTbxq'},
 'type': 'message_creation'}

null


> **Note**
> There are more intricacies in Retrieval, like [Annotations](https://platform.openai.com/docs/assistants/how-it-works/managing-threads-and-messages), which may be covered in another cookbook.


### Functions

As a final powerful tool for your Assistant, you can specify custom [Functions](https://platform.openai.com/docs/assistants/tools/function-calling) (much like the [Function Calling](https://platform.openai.com/docs/guides/function-calling) in the Chat Completions API). During a Run, the Assistant can then indicate it wants to call one or more functions you specified. You are then responsible for calling the Function, and providing the output back to the Assistant.

Let's take a look at an example by defining a `display_quiz()` Function for our Math Tutor.

This function will take a `title` and an array of `question`s, display the quiz, and get input from the user for each:

- `title`
- `questions`
  - `question_text`
  - `question_type`: [`MULTIPLE_CHOICE`, `FREE_RESPONSE`]
  - `choices`: ["choice 1", "choice 2", ...]

Unfortunately I don't know how to get user input within a Python Notebook, so I'll be mocking out responses with `get_mock_response...`. This is where you'd get the user's actual input.


In [ ]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

Here's what a sample quiz would look like:


In [ ]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


Now, let's define the interface of this function in JSON format, so our Assistant can call it:


In [ ]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

Once again, let's update our Assistant either through the Dashboard or the API.


![Enabling custom function](../images/assistants_overview_enable_function.png)

> **Note**
> Pasting the function JSON into the Dashboard was a bit finicky due to indentation, etc. I just asked ChatGPT to format my function the same as one of the examples on the Dashboard :).


In [ ]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'created_at': 1710933670,
 'description': None,
 'file_ids': ['file-u5H0QWd95vzYHIEmJfR1Vn3E'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE']},
 

And now, we ask for a quiz.


In [ ]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

Now, however, when we check the Run's `status` we see `requires_action`! Let's take a closer.


In [ ]:
show_json(run)

{'id': 'run_6vZeKXfIH2w7ZWFzykFaQ6Rl',
 'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710934626,
 'expires_at': 1710935226,
 'failed_at': None,
 'file_ids': ['file-u5H0QWd95vzYHIEmJfR1Vn3E'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_PTIQgLYGFNrjtCyWJAqsgX75',
     'function': {'arguments': '{"title":"Math Quiz","questions":[{"question_text":"What is the result of 5 multiplied by 7?","question_type":"FREE_RESPONSE"},{"question_text":"Which of the following is a prime number?","question_type":"MULTIPLE_CHOICE","choices":["9","11","14","20"]}]}',
      'name': 'display_quiz'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'started_at': 1710934642,
 'status': 'requires_action',
 'thread_id':

The `required_action` field indicates a Tool is waiting for us to run it and submit its output back to the Assistant. Specifically, the `display_quiz` function! Let's start by parsing the `name` and `arguments`.

> **Note**
> While in this case we know there is only one Tool call, in practice the Assistant may choose to call multiple tools.


In [ ]:
# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Quiz',
 'questions': [{'question_text': 'What is the result of 5 multiplied by 7?',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'Which of the following is a prime number?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['9', '11', '14', '20']}]}

Now let's actually call our `display_quiz` function with the arguments provided by the Assistant:


In [ ]:
responses = display_quiz(arguments["title"], arguments["questions"])
print("Responses:", responses)

Quiz: Math Quiz

What is the result of 5 multiplied by 7?

Which of the following is a prime number?
0. 9
1. 11
2. 14
3. 20

Responses: ["I don't know.", 'a']


Great! (Remember these responses are the one's we mocked earlier. In reality, we'd be getting input from the back from this function call.)

Now that we have our responses, let's submit them back to the Assistant. We'll need the `tool_call` ID, found in the `tool_call` we parsed out earlier. We'll also need to encode our `list`of responses into a `str`.


In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_6vZeKXfIH2w7ZWFzykFaQ6Rl',
 'assistant_id': 'asst_wIiQdreLlLNzoud1MxL2iheM',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1710934626,
 'expires_at': 1710935226,
 'failed_at': None,
 'file_ids': ['file-u5H0QWd95vzYHIEmJfR1Vn3E'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1710934642,
 'status': 'queued',
 'thread_id': 'thread_XdcqyZOU5RrLlfjfCKpJDicM',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and 

We can now wait for the Run to complete once again, and check our Thread!


In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: For the question "What is the result of 5 multiplied by 7?", your response was "I don't know." The correct answer is 35.

For the question "Which of the following is a prime number?", your response was "a." The correct answer is "11," which is a prime number.



Woohoo 🎉


# Thi Đấu Assistant vs GPT-3.5

## Assistant

In [ ]:
# Update Assistant
assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo",
    instructions="You are a math tutor.",
    tools=[{"type": "code_interpreter"}],
    file_ids=None
)
show_json(assistant)

{'id': 'asst_r7UpyWvElzUs3AzUmrvXUwsg',
 'created_at': 1710936429,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a math tutor.',
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'name': None,
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

In [ ]:
thread, run = create_thread_and_run("""The test has 5 questions. Answer the questions with code:
1. From a pack of 52 cards, a card is drawn at random. What is the probability of getting a queen?
2. Throw 2 dices simultaneously. What is the probability that the summation of the numbers is multiply of 4?
3. 10 coins are tossed, find the probability that two heads are obtained.
4. If P(A)=0.4, P(B)=0.6 and P(A ∪ B)=0.8. What is the value of P(A∩B')=?
5. Throw a dice 3 times. What's the probability that we have three 6?""", id=assistant.id)
run = wait_on_run(run, thread)
run.status

'completed'

In [ ]:
show_json(run)

{'id': 'run_58jEVTHf8lNkMD9v8v03NKRz',
 'assistant_id': 'asst_r7UpyWvElzUs3AzUmrvXUwsg',
 'cancelled_at': None,
 'completed_at': 1710936486,
 'created_at': 1710936471,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a math tutor.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1710936471,
 'status': 'completed',
 'thread_id': 'thread_3sCDSwKeHTTz53OcwuShCaMT',
 'tools': [{'type': 'code_interpreter'}],
 'usage': {'completion_tokens': 749,
  'prompt_tokens': 1616,
  'total_tokens': 2365}}

In [ ]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: The test has 5 questions. Answer the questions with code:
1. From a pack of 52 cards, a card is drawn at random. What is the probability of getting a queen?
2. Throw 2 dices simultaneously. What is the probability that the summation of the numbers is multiply of 4?
3. 10 coins are tossed, find the probability that two heads are obtained.
4. If P(A)=0.4, P(B)=0.6 and P(A ∪ B)=0.8. What is the value of P(A∩B')=?
5. Throw a dice 3 times. What's the probability that we have three 6?
assistant: Let's solve each of these questions one by one:

1. The probability of getting a queen from a pack of 52 cards is given by the number of queens divided by the total number of cards.
\[P(\text{Queen}) = \frac{\text{Number of Queens}}{\text{Total Number of Cards}}\]

2. To find the probability that the sum of two dice is a multiple of 4, we need to calculate the total number of favorable outcomes and divide it by the total number of outcomes when throwing two dice.

3. The probability 

In [ ]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [ ]:
run_steps.data

[RunStep(id='step_bKjQbI7xs2B6zpfu8uODNer8', assistant_id='asst_r7UpyWvElzUs3AzUmrvXUwsg', cancelled_at=None, completed_at=1710936476, created_at=1710936471, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_58jEVTHf8lNkMD9v8v03NKRz', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_idA7Hx4IIz4hNlql5O6anctH'), type='message_creation'), thread_id='thread_3sCDSwKeHTTz53OcwuShCaMT', type='message_creation', usage=Usage(completion_tokens=227, prompt_tokens=248, total_tokens=475), expires_at=None),
 RunStep(id='step_oaBeIV3SCkbuB3qATVEZy3Fo', assistant_id='asst_r7UpyWvElzUs3AzUmrvXUwsg', cancelled_at=None, completed_at=1710936484, created_at=1710936476, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_58jEVTHf8lNkMD9v8v03NKRz', status='completed', step_details=ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_I

In [ ]:
assistant_response = get_response(thread).data[-1].content[0].text.value
print(assistant_response)

Here are the probabilities for each of the questions:

1. The probability of getting a queen from a pack of 52 cards is approximately 0.077 or 7.69%.
2. The probability that the sum of two dice is a multiple of 4 is approximately 0.278 or 27.78%.
3. The probability of obtaining exactly two heads when tossing 10 coins is approximately 0.044 or 4.39%.
4. The value of \(P(A \cap B')\) is approximately 0.200 or 20%.
5. The probability of getting three 6's when throwing a dice 3 times is approximately 0.005 or 0.46%.

If you have any more questions or need further clarification, feel free to ask!


## GPT-3.5

In [ ]:
import openai
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
gpt3_5_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': """The test has 5 questions. Answer each question very short, in less than a sentence:
1. From a pack of 52 cards, a card is drawn at random. What is the probability of getting a queen?
2. Throw 2 dices simultaneously. What is the probability that the summation of the numbers is multiply of 4?
3. 10 coins are tossed, find the probability that two heads are obtained.
4. If P(A)=0.4, P(B)=0.6 and P(A ∪ B)=0.8. What is the value of P(A∩B')=?
5. Throw a dice 3 times. What's the probability that we have three 6?"""}]
)

In [ ]:
gpt3_5_response.choices[0].message.content

'1. 4/52 or 1/13\n2. 4/36 or 1/9\n3. 9/1024 or 0.0088\n4. 0.2 \n5. 1/216'